In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
def load_and_prepare_prompts(file_path, tokenizer):
    data = pd.read_csv(file_path)
    prompts = []

    eos_token = tokenizer.eos_token if tokenizer.eos_token else ""

    for _, row in data.iterrows():
        input_text = row.get("input", "")
        output_text = row.get("output", "")
        formatted_prompt = (
            "You are a helpful assistant specializing in restoring obfuscated Korean reviews.\n"
            "Your task is to transform the given obfuscated Korean review into a clear, correct,\n"
            "and natural-sounding Korean review that reflects its original meaning.\n"
            "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
            f"### Obfuscated Review:\n{input_text}\n\n"
            f"### Restored Review:\n{output_text}{eos_token}"
        )
        prompts.append(formatted_prompt)

    return prompts

In [ ]:
import pandas as pd
from datasets import Dataset
dataset_file='/content/train.csv'

In [ ]:
def create_hf_dataset(prompts):
    dataset_dict = {"text": prompts}
    hf_dataset = Dataset.from_dict(dataset_dict)
    return hf_dataset

prompts = load_and_prepare_prompts(dataset_file, tokenizer)
dataset = create_hf_dataset(prompts)

In [ ]:
dataset[0]

{'text': 'You are a helpful assistant specializing in restoring obfuscated Korean reviews.\nYour task is to transform the given obfuscated Korean review into a clear, correct,\nand natural-sounding Korean review that reflects its original meaning.\nBelow are examples of obfuscated Korean reviews and their restored forms:\n\n### Obfuscated Review:\n별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.\n\n### Restored Review:\n별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자니 댓글로는 너무 길고... 아무튼 두 번 다시 가길 싫은 곳. 캠핑을 20여 년 다녀본 곳 중 제일 기분 나빴던 곳.<|eot_id|>'}

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/11263 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,263 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.699100
2,4.614700
3,4.798800
4,4.756200
5,4.583900
6,4.674900
7,4.072400
8,3.952900
9,3.715700
10,3.295100


In [ ]:
FastLanguageModel.for_inference(model)
def restore_obfuscated_review(input_example: str, max_new_tokens):
    formatted_prompt = (
        "You are a helpful assistant specializing in restoring obfuscated Korean reviews.\n"
        "Your task is to transform the given obfuscated Korean review into a clear, correct,\n"
        "and natural-sounding Korean review that reflects its original meaning.\n"
        "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
        f"### Obfuscated Review:\n{input_example}\n\n"
        "### Restored Review:\n"
    )

    inputs = tokenizer([formatted_prompt], return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        model.cuda()

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        do_sample=False,
        temperature=0.2,
        top_p=0.8,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return decoded[0]

if __name__ == "__main__":
    test_obfuscated_review = "쫄용햐닌 좋숩뉘탸. 쩌움 옮면 죡큼 닿황할 숯토 윗엊요. 쭙변예 얌뭅걷돎 엾였석."
    test_len=len(test_obfuscated_review)
    restored_text = restore_obfuscated_review(test_obfuscated_review,test_len)
    print("=== Obfuscated Review ===")
    print(test_obfuscated_review)
    print("\n=== Restored Review ===")
    print(restored_text)

=== Obfuscated Review ===
쫄용햐닌 좋숩뉘탸. 쩌움 옮면 죡큼 닿황할 숯토 윗엊요. 쭙변예 얌뭅걷돎 엾였석.

=== Restored Review ===
You are a helpful assistant specializing in restoring obfuscated Korean reviews.
Your task is to transform the given obfuscated Korean review into a clear, correct,
and natural-sounding Korean review that reflects its original meaning.
Below are examples of obfuscated Korean reviews and their restored forms:

### Obfuscated Review:
쫄용햐닌 좋숩뉘탸. 쩌움 옮면 죡큼 닿황할 숯토 윗엊요. 쭙변예 얌뭅걷돎 엾였석.

### Restored Review:
조용하지 않네요. 전원 오면 조그만 다하고 싶어요. 그분이 안먹고도 어제서요.
